# NRM Configuration/Manifest guide

This notebook documents NRM's configuration and manifest format. The two next cells are for setup purposes.

In [1]:
%%capture
cd ..

In [2]:
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)

## Daemon configuration

`nrmd`'s configuration can be defined in the json/yaml/[Dhall](https://dhall-lang.org/) formats. Admissible values are defined in file [resources/configurationSchema.json](./resources/configurationSchema.json), and alternatively available as a Dhall type in [resources/types/Cfg.dhall](resources/types/Configuration.dhall). Schema files get large, so the next cells shows the Dhall Configuration type as a more readable alternative.

In [3]:
%%script dhall resolve
./hsnrm/resources/defaults/Cfg.dhall

{ activeSensorFrequency =
    { fromHz = 1.0 }
, argo_nodeos_config =
    "argo_nodeos_config"
, argo_perf_wrapper =
    "nrm-perfwrapper"
, controlCfg =
    < ControlCfg :
        { hint :
            < Full
            | Only :
                { only :
                    List (List { actuatorID : Text, actuatorValue : Double })
                }
            >
        , learnCfg :
            < Contextual :
                { contextual : { horizon : Integer } }
            | Lagrange :
                { lagrange : Double }
            | Random :
                { random : Optional Integer }
            >
        , minimumControlInterval :
            { fromuS : Double }
        , referenceMeasurementRoundInterval :
            Integer
        , speedThreshold :
            Double
        , staticPower :
            { fromuW : Double }
        }
    | FixedCommand :
        { fixedPower : { fromuW : Double } }
    >.FixedCommand
    { fixedPower = { fromuW = 2.5e8 } }
, downstreamCfg 

Optional values are filled using defaults that can be found in [resources/defaults/Cfg.json](resources/defaults/Configuration.json) (also available in the Dhall format):

In [4]:
%%bash
cat ./hsnrm/resources/defaults/Cfg.json | jq

{
  "pmpi_lib": "pmpi_lib",
  "verbose": "Error",
  "logfile": "/tmp/nrm.log",
  "singularity": false,
  "argo_nodeos_config": "argo_nodeos_config",
  "controlCfg": {
    "fixedPower": {
      "fromuW": 250000000
    }
  },
  "upstreamCfg": {
    "upstreamBindAddress": "*",
    "rpcPort": 3456,
    "pubPort": 2345
  },
  "libnrmPath": null,
  "activeSensorFrequency": {
    "fromHz": 1
  },
  "perf": "perf",
  "argo_perf_wrapper": "nrm-perfwrapper",
  "downstreamCfg": {
    "downstreamBindAddress": "ipc:///tmp/nrm-downstream-event"
  },
  "nodeos": false,
  "hwloc": "hwloc",
  "raplCfg": {
    "referencePower": {
      "fromuW": 250000000
    },
    "raplActions": [
      {
        "fromuW": 100000000
      },
      {
        "fromuW": 200000000
      }
    ],
    "raplPath": "/sys/devices/virtual/powercap/intel-rapl"
  },
  "dummy": true,
  "slice_runtime": "Dummy",
  "hwmonCfg": {
    "hwmonPath": "/sys/class/hwmon",
    "hwmonEnabled": true
  }
}


## Manifest configuration

Example manifest files are in [resources/examples](../resources/examples) in JSON/YAML/Dhall format. For instance, the manifest file [resources/examples/perfwrap.json](../resources/examples/perfwrap.json) enables enables performance monitoring:

In [5]:
%%bash
cat ./hsnrm/resources/examples/perfwrap.json | jq

{
  "image": null,
  "hwbind": false,
  "app": {
    "scheduler": "FIFO",
    "instrumentation": null,
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "NoPowerPolicy"
    },
    "perfwrapper": {
      "perfLimit": {
        "fromOps": 100000
      },
      "perfFreq": {
        "fromHz": 1
      }
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


Manifest options are documented in schema file [resources/manifestSchema.json](../resources/manifestSchema.json). The next cell shows the corresponding [Dhall](https://dhall-lang.org/) type.

In [6]:
%%script dhall resolve
./hsnrm/resources/types/Manifest.dhall

{ app :
    { instrumentation :
        Optional { ratelimit : { fromHz : Double } }
    , perfwrapper :
        < Perfwrapper :
            { perfFreq :
                { fromHz : Double }
            , perfLimit :
                { fromOps : Integer }
            }
        | PerfwrapperDisabled
        >
    , power :
        { policy :
            < Combined | DDCM | DVFS | NoPowerPolicy >
        , profile :
            Bool
        , slowdown :
            Integer
        }
    , scheduler :
        < FIFO | HPC | Other : { _1 : Integer } >
    , slice :
        { cpus : Integer, mems : Integer }
    }
, hwbind :
    Bool
, image :
    Optional
    { binds : Optional (List Text), imagetype : < Docker | Sif >, path : Text }
, name :
    Text
}


Under-specified manifests like the one in our `workloads` above (with missing optional fields from the schema) fill missing values with defaults, which are located in file [resources/defaults/Manifest.json](../../resources/examples/default.json):

In [7]:
%%bash
cat hsnrm/resources/defaults/Manifest.json | jq

{
  "image": null,
  "hwbind": false,
  "app": {
    "scheduler": "FIFO",
    "instrumentation": null,
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "NoPowerPolicy"
    },
    "perfwrapper": "PerfwrapperDisabled",
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


The `dhall` and `dhall-to-json` utilities are available as convenience in this environment should you need them. Dhall is useful as a configuration language in itself:

In [8]:
%%script bash
dhall-to-json <<< 'let Manifest = ./hsnrm/resources/types/Manifest.dhall 
    let appendName = \(m: Manifest) -> m // {name = m.name ++ "-appended" }
    in appendName ./hsnrm/resources/defaults/Manifest.dhall
' | jq

{
  "image": null,
  "hwbind": false,
  "app": {
    "scheduler": "FIFO",
    "instrumentation": null,
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "NoPowerPolicy"
    },
    "perfwrapper": "PerfwrapperDisabled",
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default-appended"
}


Remember that any json document is one little step away from being a Python dictionaryy: